# Running Codes

Testing Features Extraction

In [ ]:
# Code for testing the Feature Extraction Module

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)
    
    feat_extract = FeatureExtraction(method="SIFT", verbose=cmd_args.verbose)
    
    # Extracting Features from Image Map
    features = feat_extract.extract_features(parser.config_dict["image_map"], type="image")
    
    # Extracting Features from video files (Only One Video Accepted)
    features_video = feat_extract.extract_features(parser.config_dict["videos"], type="video")
    features.extend(features_video)
    
    parser.save_features(features)

Testing Features Matching

In [ ]:
# Code for testing the Feature Matching (without initially saving the features)

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)
    
    # Loading extracted features
    feat_extract = FeatureExtraction(method="SIFT", verbose=cmd_args.verbose)
    
    # Extracting Features from Image Map
    features = feat_extract.extract_features(parser.config_dict["image_map"], type="image")
    
    # Extracting Features from video files (Only One Video Accepted)
    features_video = feat_extract.extract_features(parser.config_dict["videos"], type="video")
    features.extend(features_video)
    
    # Matching Features
    feat_match = FeatureMatching(lib="sklearn", verbose=cmd_args.verbose)
    pts_in_map, pts_in_frame = feat_match.match_features(features)
    
    print(len(pts_in_map), len(pts_in_frame))


In [ ]:
# Code for testing the Feature Matching Module after Extracting Features

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)
    
    # Loading extracted features
    features = parser.load_features()
    
    # Matching Features
    feat_match = FeatureMatching(lib="sklearn", verbose=cmd_args.verbose)
    pts_in_map, pts_in_frame = feat_match.match_features(features)
    
    print(len(pts_in_map), len(pts_in_frame))

Homography

In [ ]:
# Code for calculating homography matrix from the scratch 
# (Loading video, obtaining features, matching features and calculating homographies).

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)

    # Initializing the Feature Extraction
    feat_extract = FeatureExtraction(method="SIFT", blur=True,
                                     verbose=cmd_args.verbose, visualize=cmd_args.visualize)
    
    # Extracting Features from video files (Only One Video Accepted)
    features, frame_ids = feat_extract.extract_features(parser.config_dict["videos"], type="video",
                                                              visualization_delay=1, # Visualization delay in seconds
                                                              )
    parser.save_features(features)
    
    # Matching Features
    feat_match = FeatureMatching(lib="sklearn", verbose=cmd_args.verbose, visualize=cmd_args.visualize)
    
    corr_ref_ids, pts_in_ref, pts_in_frame = feat_match.match_features(features,
                                                                       frame_ids=frame_ids,
                                                                       ref_frame_ids=parser.config_dict["frame_ids"], # Frame IDs use 1-indexing
                                                                       video_path=parser.config_dict["videos"],
                                                                       visualization_delay=1)
    
    # Computing Homography
    homography = Homography(use_ransac=True, use_opencv=False, verbose=cmd_args.verbose)
    homo_output_matrix_map = homography.compute_homography( pts_in_ref, pts_in_frame, frame_ids, corr_ref_ids, transforms = "map" )
    homo_output_matrix_ref_frames = homography.compute_homography( parser.config_dict["pts_in_map"], parser.config_dict["pts_in_frame"], 
                                                                  parser.config_dict["frame_ids"], [0]*len(parser.config_dict["frame_ids"]),
                                                                  transforms = "map")
    
    parser.save_homography_output(homo_output_matrix_map)

In [ ]:
if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)

    # Feature Extraction
    feat_extract = FeatureExtraction(method="SIFT", blur=True, verbose=cmd_args.verbose, visualize=cmd_args.visualize)
    features, frame_ids = feat_extract.extract_features(parser.config_dict["videos"], # Extracting Features from video files (Only One Video Accepted)
                                                        type="video",
                                                        visualization_delay=1, # Visualization delay in seconds
                                                        )
    parser.save_features(features)
    
    # Matching Features
    feat_match = FeatureMatching(lib="sklearn", verbose=cmd_args.verbose, visualize=cmd_args.visualize)
    corr_ref_ids, pts_in_ref, pts_in_frame = feat_match.match_features(features,
                                                                       frame_ids=frame_ids,
                                                                       ref_frame_ids=parser.config_dict["frame_ids"], # Frame IDs use 1-indexing
                                                                       video_path=parser.config_dict["videos"],
                                                                       visualization_delay=1)
    
    # Computing Homography Matrices
    homography = Homography(transforms=parser.config_dict["transforms"], use_ransac=True, use_opencv=False, verbose=cmd_args.verbose)
    homo_output_matrix = homography.compute_homography( pts_in_ref, pts_in_frame, frame_ids, corr_ref_ids, parserObject=parser)
    
    parser.save_homography_output(homo_output_matrix)


In [ ]:
# Code for calculating homography matrix using the features saved in the keypoint_out path..

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)

    # Loading extracted features
    features = parser.load_features()

    # Matching Features
    feat_match = FeatureMatching(lib="sklearn", verbose=cmd_args.verbose)
    pts_in_map, pts_in_frame = feat_match.match_features(features)
    frame_ids = list( range(1, len(pts_in_frame)+1) )

    # Computing Homography
    homography = Homography(transforms="map", use_ransac=True, verbose=cmd_args.verbose)
    homo_output_matrix = homography.compute_homography( pts_in_map, pts_in_frame, frame_ids )
    parser.save_homography_output(homo_output_matrix)

In [ ]:
# Code for calculating homography matrix from the data in the config file.

if __name__ == '__main__':
    cmd_args = parse_args() #Read arguments passed on the command line
    parser = ConfigParser(config_path = cmd_args.config_file_path, verbose=cmd_args.verbose)

    # Computing Homography
    homography = Homography(transforms="map", use_ransac=True, verbose=cmd_args.verbose)
    homo_output = homography.compute_homography(parser.config_dict["map_points"], parser.config_dict["frame_points"], parser.config_dict["frame_ids"])
    parser.save_homography_output(homo_output_matrix)

# Debugging

In [ ]:
if __name__ == '__main__':
    verbose = True
    visualize = False
    config_path = "src/myconfig.cfg"
    
    parser = ConfigParser(config_path = config_path, verbose=verbose)

    # Feature Extraction
    feat_extract = FeatureExtraction(method="SIFT", blur=True, verbose=verbose, visualize=visualize)
    features, frame_ids = feat_extract.extract_features(parser.config_dict["videos"], # Extracting Features from video files (Only One Video Accepted)
                                                        type="video",
                                                        visualization_delay=1, # Visualization delay in seconds
                                                        )
    parser.save_features(features)
    
    # Matching Features
    feat_match = FeatureMatching(lib="sklearn", verbose=verbose, visualize=visualize)
    corr_ref_ids, pts_in_ref, pts_in_frame = feat_match.match_features(features,
                                                                       frame_ids=frame_ids,
                                                                       ref_frame_ids=parser.config_dict["frame_ids"], # Frame IDs use 1-indexing
                                                                       video_path=parser.config_dict["videos"],
                                                                       visualization_delay=1)
    
    # Computing Homography Matrices
    homography = Homography(transforms=parser.config_dict["transforms"], use_ransac=True, use_opencv=False, verbose=verbose)
    homo_output_matrix = homography.compute_homography( pts_in_ref, pts_in_frame, frame_ids, corr_ref_ids, parserObject=parser)
    
    parser.save_homography_output(homo_output_matrix)
